### Import Projects from the Acknowledger System to Jekyll Site
[J. Nathan Matias](https://twitter.com/natematias), July 2018

This code imports projects from the [acknowledger system](https://civic.mit.edu/2013/02/17/saying-thanks-online-the-visual-display-of-cooperative-acknowledgment/) ([source](https://github.com/mitmedialab/acknowledger)) database and publishes them to projects in a Jekyll-academic template. 

In [214]:
import yaml, _mysql, re, frontmatter, glob, os
import MySQLdb.cursors
from dateutil import parser
from collections import defaultdict, Counter

def slugify(value):
    """
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens. modified from the Django codebase
    """
    import unicodedata
    value = re.sub('[^\w\s-]', '', value).strip().lower()
    value = re.sub('[-\s]+', '-', value)
    return value


with open("database.yml", "r") as f:
    settings = yaml.load(f.read())
    if settings['password'] is None:
        settings['password']  = ''
    
db = _mysql.connect(settings['host'], settings['username'], 
                    str(settings['password']), settings['database'])

In [79]:
db.query("SELECT * from projects;")
r = db.store_result()
all_projects = {}

cursor = r.fetch_row(how=1)
while cursor:
    all_projects[cursor[0]['id']] = cursor[0]
    cursor = r.fetch_row(how=1)

In [80]:
## DECODE TO STRING
#all_projects[100][0]['title'].decode("utf-8") 

##TODO: CONSIDER CONVERTING IMAGE LINKS TO HTTPS

## Load Acknowledgment Data

In [109]:
db.query("""
select projects.id as project_id, people.* 
FROM projects 
    JOIN project_people ON projects.id = project_people.project_id 
    JOIN people on project_people.person_id = people.id;
""")
r = db.store_result()
project_people = defaultdict(list)

cursor = r.fetch_row(how=1)
while cursor:
    project_people[cursor[0]['project_id']].append(cursor[0])
    cursor = r.fetch_row(how=1)
print("Found {0} projects and {1} project person relationships".format(
    len(project_people),
    sum([len(people) for people in project_people.values()])))

Found 75 projects and 198 project person relationships


## Load Host Data

In [108]:
db.query("""
select projects.id as project_id, hosts.* 
FROM projects 
    JOIN project_hosts ON projects.id = project_hosts.project_id 
    JOIN hosts on project_hosts.host_id = hosts.id;
""")
r = db.store_result()
project_hosts = defaultdict(list)

cursor = r.fetch_row(how=1)
while cursor:
    project_hosts[cursor[0]['project_id']].append(cursor[0])
    cursor = r.fetch_row(how=1)
print("Found {0} projects and {1} project host relationships".format(
    len(project_hosts),
    sum([len(hosts) for hosts in project_hosts.values()])))

Found 42 projects and 50 project host relationships


## Export Acknowledgments to Markdown

In [329]:
#list(all_projects.values())[16]
#list(project_people.values())[0][0]
#list(project_hosts.values())[2][0]

In [330]:
def create_markdown_from_project(project, project_people, project_hosts):
    ## EXCLUDE THINGS THAT ARE ONLY POSTS
    category = project['category'].decode("utf-8").replace("  ", " ").strip()
    if(category =="post"or category=="post featured" 
       or category=="featured post"
       or category == "post featured review"):
        return None
    
    ## SET PROJECT SUBTITLE
    subtitle = ""
    if(project['subtitle']):
        subtitle = project['subtitle'].decode("utf-8")
    
    ## SET PROJECT DATE
    project_date = str(parser.parse(project['date']).date())
    
    ## SET PROJECT URI
    uri = ""
    if(project['uri']):
        uri = project['uri'].decode("utf-8")
    
    ## SET PROJECT IMAGE
    image = ""
    if(project['image']):
        image_text = project['image'].decode("utf-8")
        if(image_text.find("<") >-1):
            r_result = re.search('src="(.*?)"',image_text)
            image = r_result.group(1)
        elif(project['image'].find("http") > -1):
            image = project['image']
        
    ## SET TITLE
    title = project['title'].decode("utf-8")
    
    ## SET CONTENT
    content = ""
    if(project['description']!= None):
        try:
            content = project['description'].decode("utf-8")
        except:
            content = str(project['description'])[2:-1]
    
    ## SET TAGS
    tags = [x for x in category.split(" ") if x!=""]
    
    
    ## CREATE FRONTMATTER POST
    post = frontmatter.Post(content = content)
    post.metadata['title']     = title
    post.metadata['date']      = project_date
    post.metadata['excerpt']   = subtitle
    post.metadata['subtitle']  = subtitle
    post.metadata['uri']      = uri
    post.metadata['image']     = image
    post.metadata['tags']      = tags

    ## SET FILENAME
    filename_title = (" ".join(title.split(" ")[0:5])).lower()
    post.metadata['filename']  = project_date + "-" + slugify(filename_title)
   
    ## SET CREATORS
    creators = []
    if(project['id'] in project_people.keys()):
        for person in project_people[project['id']]:
            creators.append(person['name'].decode("utf-8"))
    post.metadata['creators'] = creators
    
    ## SET HOSTS
    hosts = []
    if(project['id'] in project_hosts.keys()):
        for host in project_hosts[project['id']]:
            hosts.append(host['name'].decode("utf-8"))
            
    post.metadata['hosts'] = hosts
    
    
    return post



In [332]:
project_posts = []
for project in all_projects.values():
    project_post = create_markdown_from_project(
        project,project_people, project_hosts)
    if(project_post):
        project_posts.append(project_post)
print("Analyzed {0} projects, created {1} project posts".format(
      len(all_projects), len(project_posts)))

Analyzed 172 projects, created 82 project posts


# Write Projects to File

### Load All Markdown Files in \_projects
Iterate through \__projects and extract the metadata so we can avoid creating duplicates. This code should never overwrite an existing markdown data, since we want to be able to customize individual markdown files after they are initially created.

In [333]:
current_projects = {}
current_filenames = []
for filename in glob.glob(os.path.join("_portfolio", "*")):
    current_filenames.append(filename)
    with open(filename, "r") as f:
        contents = f.read()
        if(len(contents)>0):
            md = frontmatter.loads(contents)
            current_projects[md.metadata['title']] = md.metadata

In [334]:
files_written = []
files_omitted = []
for post in project_posts:
    if(post.metadata['title'] not in current_projects.keys() and
        post.metadata['filename'] not in current_filenames):
        with open(os.path.join("_portfolio", post.metadata['filename'] + ".md"), "w+") as f:
            f.write(frontmatter.dumps(post))
            files_written.append(post.metadata['filename'])
    else:
        files_omitted.append(post.metadata['filename'])

print("Wrote {0} new files".format(len(files_written)))
print("Omitted to write {0} existing files".format(len(files_omitted)))

Wrote 82 new files
Omitted to write 0 existing files
